In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import time
from pathlib import Path
import warnings 
warnings.filterwarnings('ignore')

## 기온

In [9]:
# 현재 시각의 분을 00으로 조정하여 YYYYMMDDHHMI 형태로 포맷
current_time = datetime.now().replace(minute=0).strftime('%Y%m%d%H%M')

# URL 생성
url = f'https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=TA&tm={current_time}&disp=1&authKey=u5xvP2Q0SXGcbz9kNFlxpQ'

response = requests.get(url)
result = response.text

# 텍스트 데이터 가공 및 분리
lines = result.strip().split('\n')

# 필요한 데이터만 추출하여 리스트로 저장
data = []
for line in lines:
    if not line.startswith('#'):
        values = line.split()
        data.append(values)

# 데이터프레임 생성
columns = ['YYMMDDHHMI', 'STN', 'TA', 'HMI', 'AVG', 'QCM', 'MAX', 'MI', 'MIN', 'MI']
df = pd.DataFrame(data, columns=columns)

# STN 값이 403, 417, 421, 407, 423인 행들만 추출
selected_stn = [403, 417, 421, 407, 423]
filtered_df = df[df['STN'].astype(int).isin(selected_stn)]
filtered_df = filtered_df.drop(['HMI', 'AVG', 'QCM', 'MAX', 'MI', 'MIN', 'MI'], axis=1)

# csv 파일로 저장
filtered_df.to_csv('기온_data.csv', index=False)

## 풍속

In [10]:
# 현재 시각의 분을 00으로 조정하여 YYYYMMDDHHMI 형태로 포맷
current_time = datetime.now().replace(minute=0).strftime('%Y%m%d%H%M')

# URL 생성
url = f'https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=WD&tm={current_time}&help=1&authKey=u5xvP2Q0SXGcbz9kNFlxpQ'

response = requests.get(url)
result = response.text

response = requests.get(url)
result = response.text

# 텍스트 데이터 가공 및 분리
lines = result.strip().split('\n')

# 필요한 데이터만 추출하여 리스트로 저장
data = []
for line in lines:
    if not line.startswith('#'):
        values = line.split()
        data.append(values)

# 데이터프레임 생성
columns = ['YYMMDDHHMI', 'STN', 'WD', 'WS', 'HMI', 'WD', 'WS_MAX', 'MI', 'QCM', 'WS1', 'WD1', 'WS1', 'MI1', 'QC1', 'WD', 'WS_INS', 'MI', 'QCM']
df = pd.DataFrame(data, columns=columns)

# STN 값이 403, 417, 421, 407, 423인 행들만 추출
selected_stn = [403, 417, 421, 407, 423]
filtered_df = df[df['STN'].astype(int).isin(selected_stn)]

# YYMMDDHHMI , STN, WS 열을 제외한 나머지 열 삭제
columns_to_drop = [col for col in filtered_df.columns if col not in ['YYMMDDHHMI', 'STN', 'WS']]
filtered_df = filtered_df.drop(columns=columns_to_drop)

# csv 파일로 저장
filtered_df.to_csv('풍속_data.csv', index=False)

## 습도

In [11]:
# 현재 시각의 분을 00으로 조정하여 YYYYMMDDHHMI 형태로 포맷
current_time = datetime.now().replace(minute=0).strftime('%Y%m%d%H%M')

# URL 생성
url = f'https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=HM&tm={current_time}&help=1&authKey=u5xvP2Q0SXGcbz9kNFlxpQ'

response = requests.get(url)
result = response.text

# 텍스트 데이터 가공 및 분리
lines = result.strip().split('\n')

# 필요한 데이터만 추출하여 리스트로 저장
data = []
for line in lines:
    if not line.startswith('#'):
        values = line.split()
        data.append(values)

# 데이터프레임 생성
columns = ['YYMMDDHHMI','STN','HM','HMI','AVG','QCM','MAX','MI','MIN','MI']
df = pd.DataFrame(data, columns=columns)

# STN 값이 403, 417, 421, 407, 423인 행들만 추출
selected_stn = [403, 417, 421, 407, 423]
filtered_df = df[df['STN'].astype(int).isin(selected_stn)]

# YYMMDDHHMI , STN, WS 열을 제외한 나머지 열 삭제
columns_to_drop = [col for col in filtered_df.columns if col not in ['YYMMDDHHMI', 'STN', 'HM']]
filtered_df = filtered_df.drop(columns=columns_to_drop)

# csv 파일로 저장
filtered_df.to_csv('습도_data.csv', index=False)

## 데이터 병합

In [12]:
# 각각의 파일을 불러옵니다.
temperature_df = pd.read_csv('기온_data.csv')
wind_speed_df = pd.read_csv('풍속_data.csv')
humidity_df = pd.read_csv('습도_data.csv')

# YYMMDDHHMI 열의 값을 "YY-MM-DD HH:00:00" 형식으로 변환합니다.
temperature_df['YYMMDDHHMI'] = pd.to_datetime(temperature_df['YYMMDDHHMI'], format='%Y%m%d%H%M').dt.strftime('%Y-%m-%d %H:00:00')
wind_speed_df['YYMMDDHHMI'] = pd.to_datetime(wind_speed_df['YYMMDDHHMI'], format='%Y%m%d%H%M').dt.strftime('%Y-%m-%d %H:00:00')
humidity_df['YYMMDDHHMI'] = pd.to_datetime(humidity_df['YYMMDDHHMI'], format='%Y%m%d%H%M').dt.strftime('%Y-%m-%d %H:00:00')

# 열 이름 변경
temperature_df.rename(columns={'YYMMDDHHMI': '시간', 'TA': '기온(°C)', 'STN': '관측소명'}, inplace=True)
wind_speed_df.rename(columns={'YYMMDDHHMI': '시간', 'WS': '풍속(m/s)', 'STN': '관측소명'}, inplace=True)
humidity_df.rename(columns={'YYMMDDHHMI' : '시간', 'HM' : '습도(%)', 'STN': '관측소명'}, inplace=True)

# 기준 열들을 설정합니다.
key_columns = ['관측소명', '시간']

# 기준 열들을 기준으로 데이터를 병합합니다.
merged_df = temperature_df.merge(wind_speed_df, on=key_columns).merge(humidity_df, on=key_columns, how='outer')

# STN 값을 가져옵니다.
stn_values = merged_df['관측소명'].unique()

# 열 이름
columns = ['관측소명', '시간', '수위', '기온(°C)', '풍속(m/s)', '강수량(mm)', '습도(%)']

# 고정된 관측소명 리스트와 해당 STN 값을 매칭하는 딕셔너리
fixed_stations = {
    '월계1교': 407,
    '성동교': 421,
    '마장2교': 421,
    '고척교': 423,
    '도림교': 423,
    '기아대교': 417,
    '봉은교': 403
}

# 빈 데이터프레임 생성
fixed_df = pd.DataFrame(columns=columns)

# STN 값과 관측소명을 기준으로 데이터 추출 및 추가
for station_name, stn_value in fixed_stations.items():
    station_data = merged_df[merged_df['관측소명'] == stn_value]
    station_data['관측소명'] = station_name
    fixed_df = pd.concat([fixed_df, station_data], ignore_index=True)

## 강수량, 수위

In [13]:
# 첫번째 API Key - 하천 수위
api_key = '534b4e756d736f6e37344d715a4574'

# API 엔드포인트 URL - 하천 수위
url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/ListRiverStageService/1/21/'

# 두번째 API Key - 강수량
api_key1 = '446269544f736f6e3937714b6e746a'

# API 엔드포인트 URL - 강수량
url1 = f'http://openAPI.seoul.go.kr:8088/{api_key1}/xml/ListRainfallService/1/239/'

# HTTP GET 요청 보내기
response = requests.get(url)
response1 = requests.get(url1)

# XML 파싱 함수 정의
def parse_xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    data = {}
    for child in root.findall('./row'):
        for sub_child in child:
            data.setdefault(sub_child.tag, []).append(sub_child.text)
    return data

# 요청이 성공적인지 확인 - 하천 수위
if response.status_code == 200:
    # XML 데이터를 딕셔너리로 변환
    data_dict = parse_xml_to_dict(response.text)
    # 딕셔너리를 데이터프레임으로 변환
    df = pd.DataFrame(data_dict)
else:
    print(f"Request failed with status code: {response.status_code}")
    
df = df.drop(columns=['GU_CODE','TRANSFER_TIME','CONTROL_LEVEL'])

# 요청이 성공적인지 확인 - 강수량
if response1.status_code == 200:
    # XML 데이터를 딕셔너리로 변환
    data_dict1 = parse_xml_to_dict(response1.text)
    # 딕셔너리를 데이터프레임으로 변환
    df1 = pd.DataFrame(data_dict1)
else:
    print(f"Request failed with status code: {response1.status_code}")

df1['RAINFALL10'] = pd.to_numeric(df1['RAINFALL10'], errors='coerce')

# GU_NAME 별로 RAINFALL10의 평균 계산
average_rainfall_by_gu = df1.groupby('GU_NAME')['RAINFALL10'].mean()
average_rainfall_by_gu = pd.DataFrame(average_rainfall_by_gu)

수위_df = df.merge(average_rainfall_by_gu, on='GU_NAME', how='inner')

## 반복문
def fetch_data():
    response = requests.get(url)
    response1 = requests.get(url1)
    if response.status_code == 200 and response1.status_code == 200:
        data_dict = parse_xml_to_dict(response.text)
        df = pd.DataFrame(data_dict)
        df = df.drop(columns=['GU_CODE','TRANSFER_TIME','CONTROL_LEVEL'])
        
        data_dict1 = parse_xml_to_dict(response1.text)
        df1 = pd.DataFrame(data_dict1)
        df1['RAINFALL10'] = pd.to_numeric(df1['RAINFALL10'], errors='coerce')
        average_rainfall_by_gu = df1.groupby('GU_NAME')['RAINFALL10'].mean()
        average_rainfall_by_gu = pd.DataFrame(average_rainfall_by_gu)
        수위_df = df.merge(average_rainfall_by_gu, on='GU_NAME', how='inner')

        return 수위_df
    else:
        print(f"Request failed with status codes: {response.status_code}, {response1.status_code}")
        return None

수위_df.rename(columns={'RIVERGAUGE_NAME': '관측소명', 'STORED_TIME': '시간', 'CURRENT_LEVEL': '수위','RAINFALL10':'강수량(mm)' }, inplace=True)
수위_df['시간'] = pd.to_datetime(수위_df['시간']).dt.strftime('%Y-%m-%d %H:00:00')

selected_stations = ['월계1교', '성동교', '마장2교', '고척교', '도림교', '기아대교', '봉은교']
수위_df = 수위_df[수위_df['관측소명'].isin(selected_stations)]

In [14]:
for index, row in fixed_df.iterrows():
    matching_row = 수위_df[(수위_df['관측소명'] == row['관측소명']) & (수위_df['시간'] == row['시간'])]
    if not matching_row.empty:
        fixed_df.at[index, '수위'] = matching_row.iloc[0]['수위']
        fixed_df.at[index, '강수량(mm)'] = matching_row.iloc[0]['강수량(mm)']

# 현재 시각을 "YY-MM-DD HH:MM:00" 형식으로 포맷하고 10분 단위로 조정
current_time = datetime.now()
current_time -= timedelta(minutes=current_time.minute % 10)
current_time = current_time.strftime('%Y-%m-%d %H:%M:00')

# 시간 열을 현재 시간으로 설정
fixed_df['시간'] = current_time

In [15]:
# 관측소명 리스트
station_names = fixed_df['관측소명'].unique()

# 관측소명을 기준으로 데이터프레임 분리하고 CSV 파일에 추가 모드로 저장
for station_name in station_names:
    station_data = fixed_df[fixed_df['관측소명'] == station_name]
    file_name = f'{station_name}_data.csv'
    
    # 파일이 이미 존재하는지 확인
    file_path = Path(file_name)
    if file_path.exists():
        station_data.to_csv(file_name, mode='a', header=False, index=False, encoding='cp949')
    else:
        station_data.to_csv(file_name, index=False, encoding='cp949')

print("CSV 파일에 데이터 추가 완료")

CSV 파일에 데이터 추가 완료
